In [ ]:
# Autoreload modules
%load_ext autoreload
%autoreload 2

In [ ]:
# To have access to moduels
import sys,os
sys.path.append(os.path.dirname(os.path.realpath('')) + '/Modules')

In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.tensorboard import SummaryWriter
from torchmetrics import Accuracy

from torchvision.transforms import Resize, Compose, GaussianBlur, RandomRotation, RandomChoice, RandomApply, RandomAffine, ColorJitter, RandomHorizontalFlip, RandomVerticalFlip
from dataloader.transforms import GaussianNoise

from copy import deepcopy

from colorama import Fore

from matplotlib import pyplot as plt

from transformers import ViTConfig, ViTFeatureExtractor, ViTForImageClassification

from dataloader.dataset import ADNI3Channels
from dataloader.dataloader import ADNILoader

from utils.utils import count_parameters, save_model
from utils.report import sklearn_classification_report, custom_classification_report

# Dataset and Dataloader Setup

In [ ]:
image_size = (384, 384)
resize = Resize(size=image_size)

gaussian_blur = GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2))
gaussian_noise = GaussianNoise(mean=0, std=0.05)
random_rotation = RandomRotation(degrees=2)
random_translate = RandomAffine(degrees=0, translate=(0.01, 0.01))
color_jitter_brightness = ColorJitter(brightness=0.1)
color_jitter_contrast = ColorJitter(contrast=0.1)
color_jitter_saturation = ColorJitter(saturation=0.1)
random_vertical_flip = RandomVerticalFlip(0.5)
random_horizontal_flip = RandomHorizontalFlip(0.5)

random_choice = RandomChoice([gaussian_blur,
                              gaussian_noise,
                              color_jitter_brightness,
                              color_jitter_contrast,
                              color_jitter_saturation,
                              # random_rotation,
                              # random_translate,
                              # random_vertical_flip,
                              # random_horizontal_flip
                             ])
random_transforms = RandomApply([random_choice], p=0.7)

train_transforms = Compose([resize, random_transforms])
valid_transforms = Compose([resize])
test_transforms = Compose([resize])

In [ ]:
train_ds = ADNI3Channels("../Data/Training/", transforms=train_transforms)
valid_ds = ADNI3Channels("../Data/Validation/", transforms=valid_transforms)
test_ds = ADNI3Channels("../Data/Test/", transforms=test_transforms)

In [ ]:
idx = 0
image, label = train_ds[idx]

print("Image shape:", image.shape)
print("Label:", label.item())

print("Number of training samples:", len(train_ds))
print("Number of validation samples:", len(valid_ds))
print("Number of test samples:", len(test_ds), "\n")

fig, axes = plt.subplots(ncols=3, figsize=(6, 2), dpi=300)
for i in range(3):
    axes[i].imshow(image[i, :, :])
    axes[i].axis("off");
    # print(image[i, :, :].min(), image[i, :, :].max())

In [ ]:
id2label = {0: "CN", 1: "MCI", 2: "AD"}
label2id = {"CN": 0, "MCI": 1, "AD": 2}

print(id2label[label.item()])

In [ ]:
train_batch_size = 8
valid_batch_size = 2
test_batch_size = 2

hparams = {'train_ds': train_ds,
           'valid_ds': valid_ds,
           'test_ds': test_ds,
           'train_batch_size': train_batch_size,
           'valid_batch_size': valid_batch_size,
           'test_batch_size': test_batch_size,
           'num_workers': 20,
           'train_shuffle': True,
           'valid_shuffle': False,
           'test_shuffle': False,
           'train_drop_last': True,
           'valid_drop_last': False,
           'test_drop_last': False,
          }

train_dataloader = ADNILoader(**hparams).train_dataloader()
valid_dataloader= ADNILoader(**hparams).validation_dataloader()
test_dataloader = ADNILoader(**hparams).test_dataloader()

batch = next(iter(train_dataloader))
print(batch[0].shape)
print(batch[1].shape)

# Model Development

In [ ]:
class Model(nn.Module):
    def __init__(self, num_labels=3):
        super(Model, self).__init__()
        self.resize = Resize(image_size)
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(3, 3)),
            nn.BatchNorm2d(num_features=16),
            
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=3, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(3, 3)),
            nn.BatchNorm2d(num_features=3)
        )
        
        self.vit = ViTForImageClassification.from_pretrained('google/vit-base-patch32-384',
                                                             # output_attentions=True,
                                                             num_labels=num_labels,
                                                             hidden_dropout_prob=0.1,
                                                             # attention_probs_dropout_prob=0.1,
                                                             ignore_mismatched_sizes=True)
                
    def forward(self, x):
        x = self.cnn(x)
        x = self.resize(x)
        x = self.vit(x)
        return x.logits

In [ ]:
# Selecting GPU
GPU = {0: torch.device('cuda:0'),
       1: torch.device('cuda:1'),
       2: torch.device('cuda' if torch.cuda.is_available() else 'cpu')
      }

## Single-GPU trining
device = GPU[1]
model = Model(num_labels=3).to(device)

## Multi-GPU training
# device = GPU[2]
# model = ViT(num_labels=3, pretrained=pretrained)
# model= nn.DataParallel(model)
# model.to(device);

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch32-384',
                                                        do_resize=False,
                                                        do_normalize=False)


optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-4)

class_0_freq = 140
class_1_freq = 160
class_2_freq = 160
weight = torch.tensor([1/class_0_freq, 1/class_1_freq, 1/class_2_freq]).to(device)
criterion = nn.CrossEntropyLoss()

accuracy = Accuracy()
writer = SummaryWriter()
scheduler = ExponentialLR(optimizer, gamma=0.999)

In [ ]:
epochs = 100
train_accs = []
valid_accs = []
train_losses = []
valid_losses = []
best_loss = 100
best_acc = 0
saved = False
patience = 0
early_stop = 10

for epoch in range(epochs):
    print(Fore.YELLOW + f"Epoch: {(epoch+1):02}/{epochs}")
    for step, (x, y) in enumerate(train_dataloader):
        x = np.split(np.array(x), train_batch_size)
        for i in range(len(x)):
            x[i] = np.squeeze(x[i])
        x = torch.tensor(np.stack(feature_extractor(x)['pixel_values'], axis=0))
        x, y  = x.to(device), y.to(device)
        logits = model(x)
        criterion.weight = weight
        loss = criterion(logits, y)
        preds = logits.argmax(1)
        acc = accuracy(y.cpu(), preds.cpu())
        optimizer.zero_grad()           
        loss.backward()                 
        optimizer.step()
        train_losses.append(loss.item())
        train_accs.append(acc.item())
    
        if (step % 10 == 0) or (step == len(train_dataloader)):
            train_loss = sum(train_losses)/len(train_losses)
            train_acc = sum(train_accs)/len(train_accs)
            writer.add_scalar('train_loss', train_loss, epoch * len(train_dataloader) + step)
            writer.add_scalar('train_acc', train_acc, epoch * len(train_dataloader) + step)
            train_losses.clear()
            train_accs.clear()
                     
            model.eval() 
            with torch.no_grad():
                for x, y in valid_dataloader:
                    x = np.split(np.array(x), valid_batch_size)
                    for i in range(len(x)):
                        x[i] = np.squeeze(x[i])
                    x = torch.tensor(np.stack(feature_extractor(x)['pixel_values'], axis=0))
                    x, y  = x.to(device), y.to(device)
                    logits = model(x)
                    criterion.weight = None
                    loss = criterion(logits, y)
                    preds = logits.argmax(1)
                    acc = accuracy(y.cpu(), preds.cpu())
                    valid_losses.append(loss.item())
                    valid_accs.append(acc.item())
            
            valid_loss = sum(valid_losses)/len(valid_losses)
            valid_acc = sum(valid_accs)/len(valid_accs)
            writer.add_scalar('valid_loss', valid_loss, epoch * len(train_dataloader) + step)
            writer.add_scalar('valid_acc', valid_acc, epoch * len(train_dataloader) + step)
            valid_losses.clear()
            valid_accs.clear()
            
            if best_loss > valid_loss:
                best_loss = valid_loss
                best_model_loss = deepcopy(model.state_dict())
                saved = True
                
            if best_acc < valid_acc:
                best_acc = valid_acc
                best_model_acc = deepcopy(model.state_dict())
                saved = True
                
            if saved:
                print(Fore.GREEN + f"Training Loss(Accuracy): {train_loss:.2f}({train_acc:.2f}), Validation Loss(Accuracy): {valid_loss:.2f}({valid_acc:.2f})")
                saved = False
                patience = 0
            else:
                print(Fore.RED + f"Training Loss(Accuracy): {train_loss:.2f}({train_acc:.2f}), Validation Loss(Accuracy): {valid_loss:.2f}({valid_acc:.2f})")

            model.train()
    
    scheduler.step()
    
    print(Fore.YELLOW + "=" * 74)
    
    if patience >= early_stop:
        print("Early stop activated!")
        break
        
    patience += 1

# Model Save and Load

In [ ]:
save_model(best_model_acc, "Best models/", "Hybrid_acc.pt")
save_model(best_model_loss, "Best models/", "Hybrid_loss.pt")

In [ ]:
model.load_state_dict(torch.load("Best models/Hybrid_loss.pt"))

# Evaluation

In [ ]:
train_transforms = Compose([resize])

train_ds = ADNI3Channels("../Data/Training/", transforms=train_transforms)

hparams['train_ds'] = train_ds
hparams['train_shuffle'] = False

train_dataloader = ADNILoader(**hparams).train_dataloader()

In [ ]:
def predict(model, dataloader, device):
    y_true = []
    y_pred = []
    
    model.eval()
    with torch.no_grad():
        for step, (x, y) in enumerate(dataloader):
            x = np.split(np.array(x), dataloader.batch_size)
            for i in range(len(x)):
                x[i] = np.squeeze(x[i])
            x = torch.tensor(np.stack(feature_extractor(x)['pixel_values'], axis=0))
            x, y  = x.to(device), y.to(device)
            logits = model(x)
            preds = logits.argmax(1)
        
            y_pred.append(preds.cpu().numpy())
            y_true.append(y.cpu().numpy())

    y_pred = np.concatenate(y_pred, axis=0)
    y_true = np.concatenate(y_true, axis=0)
    
    return y_true, y_pred

y_true, y_pred = predict(model, valid_dataloader, device)
custom_classification_report(y_true, y_pred)
sklearn_classification_report(y_true, y_pred)
print('#'*53)

y_true, y_pred = predict(model, test_dataloader, device)
custom_classification_report(y_true, y_pred)
sklearn_classification_report(y_true, y_pred)
print('#'*53)